In [22]:
# todo
# 16/3/23

The idea is to make some surface plots over the TD space to compare:
            resolution degradation for a fixed minimum AASR,
            swath portion over which RASR excedes a certain treshold,
            peak core snr
            NEXT what happens morphing the antenna size?

In [23]:
# From Michelangelo Villano's hands on lab
# To open separate plot-windows outside the browser uncomment one of the following two lines
%matplotlib qt
#get_ipython().run_line_magic('matplotlib','qt5')

# To open a Plot-window within notebook with zoom/edit control uncomment one of the following two lines
# %matplotlib notebook
# get_ipython().run_line_magic('matplotlib','notebook')

# options are 'osx', 'qt4', 'qt5', 'gtk3', 'wx', 'qt', 'gtk', 'tk' , 'notebook' , 'inline'

In [24]:
## Dependencies
from libs.timing_diagram import *
from libs.design_functions import *
import numpy as np
import matplotlib.pyplot as plt
from libs.spherical_earth_geometry_radar import *
from libs.radartools.farField import UniformAperture
from libs.ambiguity_functions import *
from unambiguous_imaging_design_analysis import *

_*Problem : how to sample the space:*_

**possibility 1** we restrict the sampling space to solutions which align at the far range (transmit event excluded) with the next nadir return.
- few points have to be computed
- some dynamics might be hidden

**possibility 2** we set a minimum and maximum range, we discretize the PRF axis, and we find all possible ranges for each sample within the swath.
- resolution set by the user
- the samples might not coincide with the nadir return avoidance optimal (a visual selection can still be made in any case.

**2** is better overall

Also we can use the intersection point of the two minprf and opt prf curves as a reference point

In [25]:
## System Parameters

# constants
c = 299792458

# radar parameters
dutycycle = 0.25  # duty cycle
h = 500e3  # height
wavelength = c / 10e9  # wavelength

# satellite speed
vs = orbital_speed(h)

# antenna size
Wa = 0.3  # antenna width in meters
La = 2  # antenna length in meters

# nadir duration in fractions of PRI for visualization in timing diagram
# note, it makes sense to use this fractional quantity as the nadir duration, if unfocused (e.g. saturated receiver), will be proportional to the impulse on time.
nadir_duration = 2 * dutycycle


In [26]:
## base diagram
# PRF axis
PRI = 1 / 7050
prf = np.linspace(1 / PRI - 4000, 1 / PRI + 4000, 100)

# plot
fig, ax = plt.subplots(1)
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False)
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h)
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)

C:\Users\menca\PycharmProjects\systemtradeoffs\libs\design_functions.py:42: RuntimeWarning: invalid value encountered in arccos
  beta = arccos(((re + h) ** 2 - re ** 2 + rs ** 2) / (2 * (re + h) * rs))
C:\Users\menca\PycharmProjects\systemtradeoffs\libs\design_functions.py:43: RuntimeWarning: invalid value encountered in arccos
  alpha = arccos(((re + h) ** 2 + re ** 2 - rs ** 2) / (2 * (re + h) * re))


(100.0, 300.0)

In [27]:
## canonical constraints line plots
# 1 ground range axis in m
ground_range = np.linspace(0, 2000, 500) * 1000

prf_opt = optimal_prf_line(ground_range, Wa, wavelength, h)
# Add to the plot
ax.plot(prf_opt, ground_range / 1000, 'red')

prf_min = minimum_prf_line(ground_range, vs, La, h)
ax.plot(prf_min, ground_range / 1000, 'green')

C:\Users\menca\PycharmProjects\systemtradeoffs\unambiguous_imaging_design_analysis.py:56: RuntimeWarning: divide by zero encountered in divide
  prf_opt = 1 / pri


In [28]:
# find the intersection point
tosolve = lambda rgx: optimal_prf_line(rgx, Wa, wavelength, h) - minimum_prf_line(rgx, vs, La, h)

rg_point = fsolve(tosolve, 200e3)
prf_point = minimum_prf_line(rg_point, vs, La, h)

print(prf_point, rg_point)

ax.plot(prf_point, rg_point / 1000,'x')

[7058.91598277] [197427.8613883]


Step 0 definition of